<a href="https://colab.research.google.com/github/gopal2812/mlblr/blob/master/adaboostalgorithmimplementation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')
import pandas as pd
!ls -al 'gdrive/My Drive/Ex_Files_Machine_Learning_EssT_ValueEstimate/Exercise Files/Chapter 3'
root_path = 'gdrive/My Drive/Ex_Files_Machine_Learning_EssT_ValueEstimate/'  #change dir to your project folder
data = pd.read_csv('gdrive/My Drive/Ex_Files_Machine_Learning_EssT_ValueEstimate/Exercise Files/Chapter 3/ml_house_data_set.csv')
data.describe()

In [0]:
# Create a web page view of the data for easy viewing
html = data[0:100].to_html()


In [0]:
import os
# Save the html to a temporary file
with open("data.html", "w") as f:
    f.write(html)

# Open the web page in our web browser
full_filename = os.path.abspath("data.html")
#webbrowser.open("file://{}".format(full_filename))

In [0]:
data.dtypes

In [0]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn import ensemble
from sklearn.metrics import mean_absolute_error
from sklearn.externals import joblib

# Remove the fields from the data set that we don't want to include in our model
del data['house_number']
del data['unit_number']
del data['street_name']
del data['zip_code']

# Replace categorical data with one-hot encoded data
features_df = pd.get_dummies(data, columns=['garage_type', 'city'])

# Remove the sale price from the feature data
del features_df['sale_price']

# Create the X and y arrays
X = features_df.as_matrix()
y = data['sale_price'].as_matrix()


In [0]:
# Split the data set in a training set (70%) and a test set (30%)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

Train the value estimator
- [Instructor] Open up train_model part 3.py. Let's create and train our machine learning model. We're going to use scikit-learn's gradient boosting regressor. Creating the gradient boosting regressor model is just one line of code. It's called gradient boosting regressor because we're using the gradient boosting algorithm and it's building a model for value prediction. Another name for value prediction is regression. We also need to set the hyper-parameters that control how the gradient boosting regressor model will run. There are a lot of hyper-parameters for this model. We'll start by using reasonable guesses for each but later we'll learn how to tune these parameters to get the best result. First, n_estimators tells the model how many decision trees to build. Higher numbers usually allow the model to be more accurate but it increases the amount of time required to run the model. We'll start with 1000. Learning rate controls how much each additional decision tree influences the overall prediction. Lower rates usually lead to higher accuracy but only works if we have n_estimators set to a high value. Let's start with 0.1 as reasonable default. Max_depth controls how many layers deep each individual decision tree can be. We'll start with 6 which means that each decision tree in the model can be up to 6 layers deep. Min_samples_leaf controls how many times a value must appear in our training set for a decision tree to make a decision based on it. Let's set it to 9. We are saying that at least 9 houses must exhibit the same characteristic before we consider it meaningful enough to build a decision tree around it. This helps prevent single outliers from influencing the model too much. Max_features is the percentage of features in our model that we randomly choose to consider each time we create a branch in our decision tree. 0.1 is a good starting place. Loss controls how scikit-learn calculates the model's error rate or cost as it learns. The huber function does a good job while not being too influenced by outliers in the data set. Next, we tell the model to train using our training data set by calling scikit-learn's fit function on the model. This is just a single line of code. A great feature of scikit-learn is that it uses the same interface for all different supervised learning models it provides. The only difference is that different models have different hyper-parameters to set. This makes it easy to experiment with different types of machine learning models to find the one that works best for your application. Finally, we'll save our train machine learning model to a file using scikit-learn's joblib.dump function. You just pass in the model and the file name and it saves it to file. This will let us use our train model again from a different program so that we can use it to make predictions for new houses. Now, we run the program to see that our model trains correctly. Right-click and Run. Depending on the speed of your computer, it might take a few minutes for training to complete. When the training finishes, the new trained_house_classifier_model.pkl file will show up on the list on the left.

In [0]:
# Fit regression model
model = ensemble.GradientBoostingRegressor(
    n_estimators=1000,
    learning_rate=0.1,
    max_depth=6,
    min_samples_leaf=9,
    max_features=0.1,
    loss='huber',
    random_state=0
)
model.fit(X_train, y_train)

# Save the trained model to a file so we can use it in other programs
joblib.dump(model, 'trained_house_classifier_model.pkl')


Measure accuracy with mean absolute error
- [Instructor] After training a machine learning model, the next step is to measure how well the model performs. Let's open up train_model pt4.py. To check the accuracy of our model's predictions, we'll use a measure called mean absolute error. Mean absolute error looks at every prediction our model makes, and it gives us an average of how wrong it was across all the predictions. Scikit-learn provides a simple mean absolute error function that we can use to do this. To use it, we pass in the y values, or the correct answers for our training data set. Then we call model.predict on X, our training features. This will generate a prediction using our training model for each entry in our training data set. Scikit-learn will compare the predictions to the expected answers and tell us how close we are. Now let's do the exact same calculation for our test data set. The only difference here is to make sure to pass in the test data instead of the training data. Let's run this and see the result. Right-click, choose Run. For the training set, the mean absolute error is $48,727. That means our model was able to predict the value of every house in our training data set to within $48,000 of the real price. Considering the wide range of houses in our model, that's pretty good. For the test set, the mean absolute error was a bit higher at $59,225. This tells us that our model still works for houses it has never seen before, but not quite as well as for the training houses. The difference in these two numbers tells us a lot about how well the model is working.

In [0]:
# Find the error rate on the training set
mse = mean_absolute_error(y_train, model.predict(X_train))
print("Training Set Mean Absolute Error: %.4f" % mse)

# Find the error rate on the test set
mse = mean_absolute_error(y_test, model.predict(X_test))
print("Test Set Mean Absolute Error: %.4f" % mse)


The brute force solution: Grid search
Two common problems in machines learning models are over fitting and under fitting. We can often fix either problem by adjusting the hyperparameters on our model. The problem is that machine learning models have lots of hyperparameters to adjust. Often the best way to find the best settings is just through trial and error, but it can take a lot of work to try all the possible combinations. Let's open up train_model.py. When we create our model, we pass in these parameters. We have six different parameters here that we can tune and most of these parameters accept any number, so we literally have an infinite number of combinations we could try. A solution for this problem is to use a grid search. A grid search is where you list out a range of settings you want to try for each parameter, and you literally try them all. You train and test the model for every combination of parameters. The combination of parameters that generates the best predictions are the set of parameters you should use for your real model. Luckily, scikit-learn totally automates this process. Let's open up grid_search.py. This is almost exactly the same code as in train_model.py. The first difference is that we declare our model without passing in any parameters. Instead, we have a param grid below. The param grid has an array for each parameter. For each setting, we add the range of values that we want to try. The ranges we have here are good values to try for most problems. A good strategy is to try a few values for each parameter, where it increases or decreases by a significant amount, like 1.0 to 0.3 to 0.1, like we have here. There's not much point in trying values that are very close, like 1.0 to 0.95, since the results probably won't be that much different. Next, define the grid search using the grid search CV function. This takes in the model object, the param grid, and the number of CPUs we want to use to run our grid search. If you have a computer with more than one CPU, you can speed things up by using all of them. Next, we call fit on the grid search object to run the grid search. It's very important that we only pass the training data into the grid search CV function. We don't give it access to our test data set. The CV in grid search CV stands for cross-validation. The function will automatically slice up the training data into smaller subsets and use part of the data for training different models and a different part of the data for testing those models. This means that the model configuration's done without ever seeing our test data. It keeps our test data totally hidden to make sure we are doing a completely blind test of the final model. Running the grid search will take a long time since it's actually training a model for every possible combination of parameters in the para grid. When it's done training, it will print out the model hyperparameters that work the best. It will also tell us the mean average error for both data sets when using the best parameters.

In [0]:
from sklearn.model_selection import GridSearchCV
# Create the model
model = ensemble.GradientBoostingRegressor()

# Parameters we want to try
param_grid = {
    'n_estimators': [500, 1000, 3000],
    'max_depth': [4, 6],
    'min_samples_leaf': [3, 5, 9, 17],
    'learning_rate': [0.1, 0.05, 0.02, 0.01],
    'max_features': [1.0, 0.3, 0.1],
    'loss': ['ls', 'lad', 'huber']
}

# Define the grid search we want to run. Run it with four cpus in parallel.
gs_cv = GridSearchCV(model, param_grid, n_jobs=4)

# Run the grid search - on only the training data!
gs_cv.fit(X_train, y_train)

# Print the parameters that gave us the best result!
print(gs_cv.best_params_)

# After running a .....long..... time, the output will be something like
# {'loss': 'huber', 'learning_rate': 0.1, 'min_samples_leaf': 9, 'n_estimators': 3000, 'max_features': 0.1, 'max_depth': 6}

# That is the combination that worked best.

# Find the error rate on the training set using the best parameters
mse = mean_absolute_error(y_train, gs_cv.predict(X_train))
print("Training Set Mean Absolute Error: %.4f" % mse)

# Find the error rate on the test set using the best parameters
mse = mean_absolute_error(y_test, gs_cv.predict(X_test))
print("Test Set Mean Absolute Error: %.4f" % mse)



- [Instructor] Let's open up feature_selection.py. In our house price model, if we include the 18 original features, plus the new features that were created by using one hot in coding, we have a total of 63 features. Some of the features, like the size of the house in sq feet, are probably really important to determining the value of the house. Other features, like whether the house has a fireplace, probably matter less when calculating the final price, but how much less? Maybe there are features that don't matter at all, and we can just remove them from our model. With the tree based machine learning algorithm like radiant boosting, we can actually look at the train model and have it tell us how often each feature is used in determining the final price. First, let's load up the model using joblib.load. If you don't have a train house classifier model.pkl file, just open up train_model.py and run it first to create one. Now we can get the importance of each feature from our trained model. To do that, we call model.feature importances ending with an underscore. In scikit-learn, this will give us an array containing the feature importance for each feature. The total of all feature importances will add up to one, so you can think of this as a percentage rating of how often the feature is used in determining a house's value. To make the list of features easier to read, let's sort them from most important to least important. We'll use numpy's argsort function to give the list of array indexes pointing to each element in the array in order. Then we'll use a forward loop to print out each feature name and how important it is. Let's run the program; right click and choose run. Here at the bottom, we can see that these last few features are the most important in the house's price. The most important factors in determining the house's price is the size, the year it was built, the size of the garage, and the number of bedrooms and the number of bathrooms. If we scroll up a little bit, we can see that other factors, like how many stories the house has, or whether or not it has a pool can affect the price, but they are less often important. If you go all the way to the top of the list, we can see that some factors, like whether or not the house is in the city of New Michele don't matter to the price at all. But overall, in our case, most of these 63 features are used, but if you have a really big model with hundreds or thousands of features, you can use this approach to choose which features to keep and which features might be better to throw out the next time you retrain it. Even if you don't end up excluding any features from your model, this is a good way of getting an idea of what your model is actually doing. For example, if you saw that the model thought that the most important features were ones you knew didn't matter, that would tell you that your model isn't working very well yet.

In [0]:
import numpy as np
from sklearn.externals import joblib

# These are the feature labels from our data set
feature_labels = np.array(['year_built', 'stories', 'num_bedrooms', 'full_bathrooms', 'half_bathrooms', 'livable_sqft', 'total_sqft', 'garage_sqft', 'carport_sqft', 'has_fireplace', 'has_pool', 'has_central_heating', 'has_central_cooling', 'garage_type_attached', 'garage_type_detached', 'garage_type_none', 'city_Amystad', 'city_Brownport', 'city_Chadstad', 'city_Clarkberg', 'city_Coletown', 'city_Davidfort', 'city_Davidtown', 'city_East Amychester', 'city_East Janiceville', 'city_East Justin', 'city_East Lucas', 'city_Fosterberg', 'city_Hallfort', 'city_Jeffreyhaven', 'city_Jenniferberg', 'city_Joshuafurt', 'city_Julieberg', 'city_Justinport', 'city_Lake Carolyn', 'city_Lake Christinaport', 'city_Lake Dariusborough', 'city_Lake Jack', 'city_Lake Jennifer', 'city_Leahview', 'city_Lewishaven', 'city_Martinezfort', 'city_Morrisport', 'city_New Michele', 'city_New Robinton', 'city_North Erinville', 'city_Port Adamtown', 'city_Port Andrealand', 'city_Port Daniel', 'city_Port Jonathanborough', 'city_Richardport', 'city_Rickytown', 'city_Scottberg', 'city_South Anthony', 'city_South Stevenfurt', 'city_Toddshire', 'city_Wendybury', 'city_West Ann', 'city_West Brittanyview', 'city_West Gerald', 'city_West Gregoryview', 'city_West Lydia', 'city_West Terrence'])

# Load the trained model created with train_model.py
model = joblib.load('trained_house_classifier_model.pkl')

# Create a numpy array based on the model's feature importances
importance = model.feature_importances_

# Sort the feature labels based on the feature importance rankings from the model
feauture_indexes_by_importance = importance.argsort()

# Print each feature label, from most important to least important (reverse order)
for index in feauture_indexes_by_importance:
    print("{} - {:.2f}%".format(feature_labels[index], (importance[index] * 100.0)))
